# 1. Install Dependencies and Setup

In [ ]:
!pip list

In [ ]:
import tensorflow as tf
import os
import sklearn

In [ ]:
# Avoid OOM errors by setting GPU Memory Consumption Growth
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus: 
    tf.config.experimental.set_memory_growth(gpu, True)

In [ ]:
tf.config.list_physical_devices('GPU')

# 2. Remove dodgy images

In [ ]:
import cv2
import imghdr

In [ ]:
x = 'area4' 

data_dir = x 

In [ ]:
image_exts = ['jpeg','jpg', 'png']

In [ ]:
import shutil

augmented_dataset_dir = 'augmented_dataset2'

if os.path.exists(augmented_dataset_dir):
    shutil.rmtree(augmented_dataset_dir)

shutil.copytree(data_dir, augmented_dataset_dir)

for image_class in os.listdir(data_dir): 
    for image in os.listdir(os.path.join(data_dir, image_class)):
        image_path = os.path.join(data_dir, image_class, image)
        try: 
            img = cv2.imread(image_path)
            tip = imghdr.what(image_path)
            if tip not in image_exts: 
                print('Image not in ext list {}'.format(image_path))
                os.remove(image_path)
        except Exception as e: 
            print('Issue with image {}'.format(image_path))
            os.remove(image_path)

# 3. Load Data

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
data = tf.keras.utils.image_dataset_from_directory(x)

In [ ]:
data_iterator = data.as_numpy_iterator()

In [ ]:
batch = data_iterator.next()

In [ ]:
fig, ax = plt.subplots(ncols=10, figsize=(100,100))
for idx, img in enumerate(batch[0][:10]):
    ax[idx].imshow(img.astype(int))
    ax[idx].title.set_text(batch[1][idx])

# 4. Scale Data

In [ ]:
data = data.map(lambda x,y: (x/255, y))

In [ ]:
data.as_numpy_iterator().next()

# 5. Split Data

In [ ]:
train_size = int(len(data)*.7)
val_size = int(len(data)*.15)
test_size = int(len(data)*.15)

In [ ]:
train_size

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# Create ImageDataGenerator instance
data_gen = ImageDataGenerator(rotation_range=20,
                              width_shift_range=0.1,
                              height_shift_range=0.1,
                              shear_range=0.1,
                              zoom_range=0.0,
                              horizontal_flip=True,
                              fill_mode='nearest')

train_gen = data_gen.flow_from_directory(data_dir,
                                         target_size=(256, 256),
                                         batch_size=32,
                                         class_mode='binary',
                                         save_to_dir='augmented_images',  # This will save augmented images to the specified folder.
                                         save_prefix='aug_',  # This will add a prefix to the augmented images' filenames.
                                         save_format='jpeg')  # This will save the images in JPEG format.

In [ ]:
def save_augmented_images(images, labels, save_to_dir, save_prefix, save_format):
    for i, (image, label) in enumerate(zip(images, labels)):
        class_folder = os.path.join(save_to_dir, str(label))
        if not os.path.exists(class_folder):
            os.makedirs(class_folder)
        file_name = f"{save_prefix}_{i}.{save_format}"
        file_path = os.path.join(class_folder, file_name)
        cv2.imwrite(file_path, image)
num_images_to_generate = len(os.listdir(data_dir)) * 20

In [ ]:
for i in range(num_images_to_generate):
    images, labels = next(train_gen)
    save_augmented_images(images, labels, augmented_dataset_dir, 'aug_', 'jpeg')

In [ ]:
data_dir = augmented_dataset_dir

In [ ]:
data = tf.keras.utils.image_dataset_from_directory(data_dir)
data_iterator = data.as_numpy_iterator()


In [ ]:
for i in range(num_images_to_generate):
    next(train_gen)

def save_augmented_images(images, labels, save_to_dir, save_prefix, save_format):
    for i, (image, label) in enumerate(zip(images, labels)):
        class_folder = os.path.join(save_to_dir, str(label))
        if not os.path.exists(class_folder):
            os.makedirs(class_folder)
        file_name = f"{save_prefix}_{i}.{save_format}"
        file_path = os.path.join(class_folder, file_name)
        cv2.imwrite(file_path, image)
num_images_to_generate = len(os.listdir(data_dir)) * 5

for i in range(num_images_to_generate):
    images, labels = next(train_gen)
    save_augmented_images(images, labels, 'augmented_images', 'aug_', 'jpeg')
    

data_dir = 'augmented_images'

In [ ]:
train = data.take(train_size)
val = data.skip(train_size).take(val_size)
test = data.skip(train_size+val_size).take(test_size)

# 6. Build Deep Learning Model

In [ ]:
train

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from tensorflow.keras import regularizers

In [ ]:
model = Sequential()

In [ ]:
#model.add(Conv2D(16, (3,3), 1, activation='relu', input_shape=(256,256,3)))
#model.add(MaxPooling2D())
#model.add(Conv2D(32, (3,3), 1, activation='relu'))
#model.add(MaxPooling2D())
#model.add(Conv2D(64, (3,3), 1, activation='relu'))
#model.add(MaxPooling2D())


#model.add(Flatten())
#model.add(Dense(512, activation='relu'))
#model.add(Dropout(0.5))

#model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.add(Conv2D(32, (5,5), 1, activation='relu', input_shape=(256,256,3)))
model.add(MaxPooling2D())
model.add(Conv2D(64, (5,5), 1, activation='relu'))
model.add(MaxPooling2D())
model.add(Conv2D(128, (5,5), 1, activation='relu'))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.001)))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.compile('adam', loss=tf.losses.BinaryCrossentropy(), metrics=['accuracy'])

In [ ]:
model.summary()

# 7. Train

In [ ]:
logdir='logs'

In [ ]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
#hist = model.fit(train, epochs=20, validation_data=val, callbacks=[tensorboard_callback])
earlystop_callback = EarlyStopping(monitor='val_loss', patience=3, verbose=1)
#earlystop_callback = EarlyStopping(monitor='val_loss', patience=15, min_delta=0.01, mode='min', verbose=1)

# Train the model with EarlyStopping callback
checkpoint_callback = ModelCheckpoint(filepath='models/area1/A13E{epoch:02d}.h5',monitor='val_loss',save_best_only=True,save_weights_only=False, mode='min',verbose=1)
hist = model.fit(train, epochs=50, validation_data=val, callbacks=[tensorboard_callback, checkpoint_callback])
                                      

# 8. Plot Performance

In [ ]:
#Training and Validation Loss: Plot the training and validation loss as a function of epochs to see how well the model is learning from the data and to identify any overfitting or underfitting issues.
plt.plot(hist.history['loss'], label='Training Loss')
plt.plot(hist.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.show()


In [ ]:
#Training and Validation Accuracy: Plot the training and validation accuracy as a function of epochs to understand how accurately the model is predicting the outcomes.
plt.plot(hist.history['accuracy'], label='Training Accuracy')
plt.plot(hist.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()
plt.show()


In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import seaborn as sns
# Get true labels and predicted labels for the test set
y_true = []
y_pred = []

for batch in test.as_numpy_iterator():
    X, y = batch
    yhat = model.predict(X)
    y_pred.extend([1 if prediction > 0.5 else 0 for prediction in yhat])
    y_true.extend(y)

# Calculate the confusion matrix
cm = confusion_matrix(y_true, y_pred)

# Visualize the confusion matrix using Seaborn's heatmap
plt.figure(figsize=(6, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Not Bump', 'Bump'], yticklabels=['Not Bump', 'Bump'])
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix for Test Data')
plt.show()

# Print the classification report
print(classification_report(y_true, y_pred, target_names=['Not Bump', 'Bump']))


In [ ]:
from sklearn.metrics import roc_curve, auc, precision_recall_curve, confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay, PrecisionRecallDisplay, RocCurveDisplay
import matplotlib.pyplot as plt
import numpy as np


In [ ]:
y_pred = model.predict(test)
y_true = np.concatenate([y for x, y in test], axis=0)


In [ ]:
fpr, tpr, thresholds = roc_curve(y_true, y_pred)
roc_auc = auc(fpr, tpr)

roc_display = RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=roc_auc, estimator_name='CNN Model')
roc_display.plot()
plt.title('ROC Curve')
plt.show()


In [ ]:
precision, recall, thresholds = precision_recall_curve(y_true, y_pred)

pr_display = PrecisionRecallDisplay(precision=precision, recall=recall, estimator_name='CNN Model')
pr_display.plot()
plt.title('Precision-Recall Curve')
plt.show()


In [ ]:

threshold = 0.5
y_pred_class = (y_pred > threshold).astype(int)
conf_matrix = confusion_matrix(y_true, y_pred_class)

cm_display = ConfusionMatrixDisplay(conf_matrix, display_labels=['Class 0', 'Class 1'])
cm_display.plot(cmap=plt.cm.Blues)
plt.title('Confusion Matrix')
plt.show()

# 9. Evaluate

In [ ]:
from tensorflow.keras.metrics import Precision, Recall, BinaryAccuracy

In [ ]:
pre = Precision()
re = Recall()
acc = BinaryAccuracy()

In [ ]:
for batch in test.as_numpy_iterator(): 
    X, y = batch
    yhat = model.predict(X)
    pre.update_state(y, yhat)
    re.update_state(y, yhat)
    acc.update_state(y, yhat)

In [ ]:
print(pre.result(), re.result(), acc.result())

# 10. Test

In [ ]:
import cv2

In [ ]:
img = cv2.imread('1.jpg')
plt.imshow(img)
plt.show()

In [ ]:
resize = tf.image.resize(img, (256,256))
plt.imshow(resize.numpy().astype(int))
plt.show()

In [ ]:
yhat = model.predict(np.expand_dims(resize/255, 0))

In [ ]:
yhat

In [ ]:
if yhat > 0.5: 
    print(f'Predicted class is Bump')
else:
    print(f'Predicted class is not a Bump')

# 11. Save the Model

In [ ]:
from tensorflow.keras.models import load_model
import tensorflow as tf
import os
import cv2
import imghdr
import numpy as np
from matplotlib import pyplot as plt
import sqlite3

In [ ]:
model.save(os.path.join('models','area2.h5'))

In [ ]:
new_model = load_model(os.path.join('models','onlysim1.h5'))

In [ ]:


# Load each image in the directory and insert it into the database if it meets the threshold
conn = sqlite3.connect('BUMPBASE.db')
conn.execute('''
    CREATE TABLE IF NOT EXISTS images (
        id INTEGER PRIMARY KEY,
        image BLOB,
        gps TEXT,
        pothole BOOLEAN
    );
''')

In [ ]:


# Specify the directory path
directory_path = "C:\\Users\\Mrjud\\Desktop\\project\\controllers\\ras\\bump_frames"

# Use the os module to get a list of all the image file names in the directory
image_names = os.listdir(directory_path)

# Load each image in the directory into a dictionary with a dynamic variable name
image_dict = {}
for i, image_name in enumerate(image_names):
    if image_name.endswith(".jpg"):
        image_path = os.path.join(directory_path, image_name)
        img = cv2.imread(image_path)
        # Set the variable name based on the index value
        variable_name = f"img{i+1}"
        image_dict[variable_name] = img
        
        # Access the first image using the img1 variable name
img1 = image_dict["img1"]

img = image_dict["img3"]
plt.imshow(img)
plt.show()



In [ ]:
#img = cv2.imread('img1.jpg')
#plt.imshow(img)
#plt.show()

In [ ]:
resize = tf.image.resize(img, (256,256))
plt.imshow(resize.numpy().astype(int))
plt.show()

In [ ]:
xhat = new_model.predict(np.expand_dims(resize/255, 0))

In [ ]:
xhat

In [ ]:
if xhat > 0.6: 
    print(f'Predicted class is Bump')
else:
    print(f'Predicted class is not a Bump')

In [ ]:
output_directory_path = "C:\\Users\\Mrjud\\Desktop\\project\\controllers\\ras\\potimg"
if not os.path.exists(output_directory_path):
    os.makedirs(output_directory_path)

In [ ]:
if xhat > 0.5:
            output_image_path = os.path.join(output_directory_path, image_name)
            cv2.imwrite(output_image_path, img)
#if xhat < 0.5:
 #           output_image_path = os.path.join(output_directory_path, image_name)
  #          cv2.imwrite(output_image_path, img)

In [ ]:
from tensorflow.keras.models import load_model
import tensorflow as tf
import os
import cv2
import imghdr
import numpy as np
from matplotlib import pyplot as plt
import sqlite3

In [ ]:
# Specify the directory path
directory_path = "C:\\Users\\Mrjud\\Desktop\\project\\controllers\\ras\\bump_frames"

# Use the os module to get a list of all the image file names in the directory
image_names = os.listdir(directory_path)

# Load each image in the directory and insert it into the database if it meets the threshold
conn = sqlite3.connect('my_database.db')
conn.execute('''
    CREATE TABLE IF NOT EXISTS images (
        id INTEGER PRIMARY KEY,
        image BLOB,
        gps TEXT,
        pothole BOOLEAN
    );
''')

In [ ]:
new_model = load_model(os.path.join('models','onlysim.h5'))

In [ ]:
for i, image_name in enumerate(os.listdir(directory_path)):
    if image_name.endswith(".jpg"):
        image_path = os.path.join(directory_path, image_name)
        img = cv2.imread(image_path)
        resize = tf.image.resize(img, (256,256))
        xhat = new_model.predict(np.expand_dims(resize/255, 0))
        if xhat > 0.6:
            with open(image_path, 'rb') as f:
                image_data = f.read()
            conn.execute('INSERT INTO images (image, gps, pothole) VALUES (?, ?, ?)', (image_data, image_name, True))
            print(f"Inserted image {i+1} into database with pothole=True")
        else:
            conn.execute('INSERT INTO images (image, gps, pothole) VALUES (?, ?, ?)', (image_data, image_name, False))
            print(f"Inserted image {i+1} into database with pothole=False")

In [ ]:
# Commit the changes and close the connection
conn.commit()
conn.close()

In [ ]:
from tensorflow.keras.models import load_model
import tensorflow as tf
import os
import cv2
import imghdr
import numpy as np
from matplotlib import pyplot as plt
import sqlite3

In [ ]:
conn = sqlite3.connect('Test2.db')
conn.execute('''
    CREATE TABLE IF NOT EXISTS images (
        image BLOB,
        gps TEXT,
        pothole BOOLEAN
    );
''')

In [ ]:
# Load the pre-trained neural network model
new_model = load_model(os.path.join('models','onlysim1.h5'))
# Specify the directory path
directory_path = "C:\\Users\\Mrjud\\Desktop\\project\\controllers\\ras\\bump_frames"

In [ ]:
for i, image_name in enumerate(os.listdir(directory_path)):
    if image_name.endswith(".jpg"):
        image_path = os.path.join(directory_path, image_name)
        img = cv2.imread(image_path)
        resize = tf.image.resize(img, (256,256))
        xhat = new_model.predict(np.expand_dims(resize/255, 0))
        pothole = bool(xhat > 0.5)
        with open(image_path, 'rb') as f:
            image_data = f.read()
        gps_coords = os.path.splitext(image_name)[0] # remove the ".jpg" extension
        conn.execute('INSERT INTO images (image, gps, pothole) VALUES (?, ?, ?)', (image_data, gps_coords, pothole))
        print(f"Inserted image {i+1} into database with pothole={pothole}")
        # Delete the file from the directory
        os.remove(image_path)

# Commit the changes and close the connection
conn.commit()
conn.close()

In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import matplotlib.pyplot as plt
import itertools

# Load the model
model_path = os.path.join('models', 'area1', 'A13E01.h5')
model = keras.models.load_model(model_path)

# Prepare the test dataset
test_data_path = 'C:\\Users\\Mrjud\\ImageClassification\\test_data\\area1'

test_images = []
test_labels = []

for label, folder_name in enumerate(['normal', 'potholes']):
    folder_path = os.path.join(test_data_path, folder_name)
    image_names = [img_name for img_name in os.listdir(folder_path) if img_name.endswith('.jpg')]

    for image_name in image_names:
        img = cv2.imread(os.path.join(folder_path, image_name))
        img = cv2.resize(img, (256, 256))
        img = img / 255.0

        test_images.append(img)
        test_labels.append(label)

test_images = np.array(test_images)
test_labels = np.array(test_labels)

# Make predictions
predictions = model.predict(test_images)
predicted_labels = np.where(predictions > 0.5, 1, 0).flatten()

# Calculate evaluation metrics
accuracy = accuracy_score(test_labels, predicted_labels)
precision = precision_score(test_labels, predicted_labels)
recall = recall_score(test_labels, predicted_labels)
f1 = f1_score(test_labels, predicted_labels)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)

# Generate a confusion matrix
cm = confusion_matrix(test_labels, predicted_labels)
plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Confusion Matrix')
plt.colorbar()
tick_marks = np.arange(2)
plt.xticks(tick_marks, ['Normal', 'Pothole'], rotation=45)
plt.yticks(tick_marks, ['Normal', 'Pothole'])

thresh = cm.max() / 2.
for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
    plt.text(j, i, cm[i, j], horizontalalignment="center", color="white" if cm[i, j] > thresh else "black")

plt.tight_layout()
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()

In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import matplotlib.pyplot as plt
import itertools

# Load the model
model_path = os.path.join('models', 'area2', 'A23E01.h5')
model = keras.models.load_model(model_path)

# Prepare the test dataset
test_data_path = 'C:\\Users\\Mrjud\\ImageClassification\\test_data\\area2'

test_images = []
test_labels = []

for label, folder_name in enumerate(['normal', 'potholes']):
    folder_path = os.path.join(test_data_path, folder_name)
    image_names = [img_name for img_name in os.listdir(folder_path) if img_name.endswith('.jpg')]

    for image_name in image_names:
        img = cv2.imread(os.path.join(folder_path, image_name))
        img = cv2.resize(img, (256, 256))
        img = img / 255.0

        test_images.append(img)
        test_labels.append(label)

test_images = np.array(test_images)
test_labels = np.array(test_labels)

# Make predictions
predictions = model.predict(test_images)
predicted_labels = np.where(predictions > 0.5, 1, 0).flatten()

# Calculate evaluation metrics
accuracy = accuracy_score(test_labels, predicted_labels)
precision = precision_score(test_labels, predicted_labels)
recall = recall_score(test_labels, predicted_labels)
f1 = f1_score(test_labels, predicted_labels)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)

# Generate a confusion matrix
cm = confusion_matrix(test_labels, predicted_labels)
plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Confusion Matrix')
plt.colorbar()
tick_marks = np.arange(2)
plt.xticks(tick_marks, ['Normal', 'Pothole'], rotation=45)
plt.yticks(tick_marks, ['Normal', 'Pothole'])

thresh = cm.max() / 2.
for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
    plt.text(j, i, cm[i, j], horizontalalignment="center", color="white" if cm[i, j] > thresh else "black")

plt.tight_layout()
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()

In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import matplotlib.pyplot as plt
import itertools

# Load the model
model_path = os.path.join('models', 'area3', 'A33E01.h5')
model = keras.models.load_model(model_path)

# Prepare the test dataset
test_data_path = 'C:\\Users\\Mrjud\\ImageClassification\\test_data\\area3'

test_images = []
test_labels = []

for label, folder_name in enumerate(['normal', 'potholes']):
    folder_path = os.path.join(test_data_path, folder_name)
    image_names = [img_name for img_name in os.listdir(folder_path) if img_name.endswith('.jpg')]

    for image_name in image_names:
        img = cv2.imread(os.path.join(folder_path, image_name))
        img = cv2.resize(img, (256, 256))
        img = img / 255.0

        test_images.append(img)
        test_labels.append(label)

test_images = np.array(test_images)
test_labels = np.array(test_labels)

# Make predictions
predictions = model.predict(test_images)
predicted_labels = np.where(predictions > 0.5, 1, 0).flatten()

# Calculate evaluation metrics
accuracy = accuracy_score(test_labels, predicted_labels)
precision = precision_score(test_labels, predicted_labels)
recall = recall_score(test_labels, predicted_labels)
f1 = f1_score(test_labels, predicted_labels)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)

# Generate a confusion matrix
cm = confusion_matrix(test_labels, predicted_labels)
plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Confusion Matrix')
plt.colorbar()
tick_marks = np.arange(2)
plt.xticks(tick_marks, ['Normal', 'Pothole'], rotation=45)
plt.yticks(tick_marks, ['Normal', 'Pothole'])

thresh = cm.max() / 2.
for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
    plt.text(j, i, cm[i, j], horizontalalignment="center", color="white" if cm[i, j] > thresh else "black")

plt.tight_layout()
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()

In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import matplotlib.pyplot as plt
import itertools

# Load the model
model_path = os.path.join('models', 'area4', 'A43E01.h5')
model = keras.models.load_model(model_path)

# Prepare the test dataset
test_data_path = 'C:\\Users\\Mrjud\\ImageClassification\\test_data\\area4'

test_images = []
test_labels = []

for label, folder_name in enumerate(['normal', 'potholes']):
    folder_path = os.path.join(test_data_path, folder_name)
    image_names = [img_name for img_name in os.listdir(folder_path) if img_name.endswith('.jpg')]

    for image_name in image_names:
        img = cv2.imread(os.path.join(folder_path, image_name))
        img = cv2.resize(img, (256, 256))
        img = img / 255.0

        test_images.append(img)
        test_labels.append(label)

test_images = np.array(test_images)
test_labels = np.array(test_labels)

# Make predictions
predictions = model.predict(test_images)
predicted_labels = np.where(predictions > 0.5, 1, 0).flatten()

# Calculate evaluation metrics
accuracy = accuracy_score(test_labels, predicted_labels)
precision = precision_score(test_labels, predicted_labels)
recall = recall_score(test_labels, predicted_labels)
f1 = f1_score(test_labels, predicted_labels)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)

# Generate a confusion matrix
cm = confusion_matrix(test_labels, predicted_labels)
plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Confusion Matrix')
plt.colorbar()
tick_marks = np.arange(2)
plt.xticks(tick_marks, ['Normal', 'Pothole'], rotation=45)
plt.yticks(tick_marks, ['Normal', 'Pothole'])

thresh = cm.max() / 2.
for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
    plt.text(j, i, cm[i, j], horizontalalignment="center", color="white" if cm[i, j] > thresh else "black")

plt.tight_layout()
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()